In [1]:
using Pkg; Pkg.activate("../")

  Activating environment at `~/research/projects/Project4.jl/Project.toml`


In [1]:
using Suppressor
@suppress begin using WGLMakie end

In [2]:
using FileIO
using Images

┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1342
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
│   exception = ErrorException("Required dependency ImageQualityIndexes [2996bd0c-7a13-11e9-2da2-2f5ce47296a9] failed to load from a cache file.")
└ @ Base loading.jl:1055
┌ Info: Precompiling ImageBase [c817782e-172a-44cc-b673-b171935fbb9e]
└ @ Base loading.jl:1342
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ImageBase [c817782e-172a-44cc-b673-b171935fbb9e]
│   exception = ErrorException("Required dependency ColorVectorSpace [c3611d14-8923-5661-9e6a-0046d554d3a4] failed to load from a cache file.")
└ @ Base loading.jl:1055
┌ Info: Precompiling ImageTransformations [02fcd773-0e25-5acc-982a-7f6622650795]
└ @ Base loading.jl:1342
┌ Warning: Module ImageBase with build ID 12020659753807 is missing from the cache.
│ This ma

In [6]:
function load_images(;path=nothing, names=[]) 
    path = isnothing(path) ? path = "../src/images/" : path 
    path = joinpath(@__DIR__, path)
    images = []
    if isempty(names)
        for i=1:8 
            push!(images, load(joinpath(path, "image0$i.jpg")))
        end
    else
        for name in names
            push!(images, load(joinpath(path, name)))
        end
    end
    return images 
end

load_images (generic function with 1 method)

In [13]:
function add_buttons!(fig)
    fig[3, 1] = buttongrid = GridLayout(tellwidth=false)
    buttonlabels = ["Previous", "Next", "Clear"]
    buttons = buttongrid[1, 1:length(buttonlabels)] = 
                [Makie.Button(fig, label=l) for l in buttonlabels]
    return buttons
end

function visualize!(images)
    fig = Figure()
    ax = WGLMakie.Axis(fig[1,1], aspect=DataAspect(), width=1000)#, limits=(-15.,15.,-15.,15.))
    hidedecorations!(ax)
    buttons = add_buttons!(fig) 
    index = 1
    frame = Observable(rotr90(images[index]))
    image!(ax, frame) 
    image_title = Observable("image01")
    Label(fig[2,1], image_title, height=5, width=20)
    points = Observable(Point2f[])
    scatter!(ax.scene, points, color=:purple)

    on(buttons[1].clicks) do i
        index = index > 1 ? index-1 : index
        frame[] = rotr90(images[index])
        image_title[] = "image0$index"
    end
    on(buttons[2].clicks) do i
        index = index < length(images) ? index+1 : index
        frame[] = rotr90(images[index])
        image_title[] = "image0$index"
    end 
    on(buttons[3].clicks) do i 
        if !isempty(points[])
            points[] = points[][1:end-1]
        end
        @show length(points[])
    end 
    on(events(ax).mousebutton) do event 
        if event.button == Mouse.left || event.button == Mouse.right
            if event.action == Mouse.press
                mp = events(ax).mouseposition[]
#                 @show mp
                push!(points[], mp)
                notify(points)
            end
        end
    end
#     on(events(ax).mouseposition) do mp
#         @show mp
#     end
    display(fig)
    return ax
end

visualize! (generic function with 1 method)

In [15]:
images = load_images()
visualize!(images)

Axis with 3 plots:
 ┣━ Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleP{3, Float32, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}}, GeometryBasics.FaceView{GeometryBasics.TriangleP{3, Float32, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}}, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, StructArrays.StructVector{GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}, NamedTuple{(:position, :normals), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}}, Int64}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}}
 ┣━ Image{Tuple{StepRangeLen{Float32, Float64, Float64}, StepRangeLen{Float32, Float64, Float64}, Matrix{RGB{Float32}}}}
 ┗━ Scatter{Tuple{Vector{Point{2, Float32}}}}
